In [2]:
# all the helper functions
from ctypes import c_uint16
def check_dual(a:list, b1:list, b2:list):
    # b1 -> even, b2 -> odd
    assert(len(a)==(len(b1)+len(b2)))
    print("[+] checking...")
    for i in range(len(b1)):
        if(a[i*2]  != b1[i] or a[i*2+1] != b2[i]):
            print(i,"a: ",a[i*2],a[i*2+1], "b: ", b1[i],b2[i])

def to_hex(a:list) -> list:
    r = []
    v = c_uint16()
    for i in a:
        v.value = i
        r.append(hex(v.value))
    return r

def string_list_conect(a:list):
    r = ""
    for i in range(len(a)):
        r = r + a[i]
    return r


In [ ]:
# checks e1 in ACCU1
# e1 = HASH -> CBD -> ACCU1
# simulation time: 36395ps ~ 77695ps when ACCU1 contains e1

from ipynb.fs.defs.cbd2 import cbd2_hash
from ipynb.fs.defs.test import convert_mem

m = '02030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f2021'
c1 = cbd2_hash(m,4)
c2 = cbd2_hash(m,5)
c3 = cbd2_hash(m,6)

print(c1)
print(c2)
print(c3)

out11 = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_a-k_0.txt")
out21 = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_a-k_1.txt")
out31 = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_a-k_2.txt")
out12 = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_b-k_0.txt")
out22 = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_b-k_1.txt")
out32 = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_b-k_2.txt")

check_dual(c1,out11,out12)
check_dual(c2,out21,out22)
check_dual(c3,out31,out32)

[1, 0, -1, 0, 1, -2, 0, 0, -1, -1, -1, -1, 0, 1, 1, -1, -1, -1, 1, 2, 0, 0, -1, 1, 0, 0, -1, 0, 1, -2, 0, -1, 0, 0, 1, -2, -1, 0, 0, -2, 1, -1, 2, 0, 1, -1, 0, 2, 0, 0, 0, 1, 0, 0, 1, -1, -1, 0, 2, 0, 2, 0, -1, 0, -1, 0, -1, -1, 1, -2, -1, 0, -1, 0, -2, 0, -1, 0, 0, 0, 0, 1, 0, 1, -1, 0, 0, 0, 0, -1, 1, -2, -1, 0, 0, 1, 1, -1, -1, 0, 0, 2, -1, 0, -1, -2, 0, 0, 1, 2, 0, 0, 1, 0, -1, 0, -1, -2, -1, -2, -1, 0, -1, 0, -1, -2, 0, 1, -1, -2, 1, 1, -1, 1, 0, 0, -1, -1, -1, 1, -2, -2, 1, -2, 1, -1, 1, 1, 0, -1, 0, 2, -1, -1, 0, 1, 1, 0, -1, 0, 0, 0, 0, -1, -1, 0, 1, -1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, -2, -1, 0, 0, -1, -2, 1, 1, 0, 1, -2, 0, 1, 1, -1, 0, -1, -1, 0, 0, 0, 2, -2, 1, -1, -1, 1, 1, 0, 1, 2, -1, -2, 0, -2, 0, 0, 0, 0, -1, 0, -2, 0, 0, 1, -1, 2, 0, -2, 0, 1, 0, 0, -1, -1, 1, 0, 0, -1, -1, 0, 1, 1, 1, 0, 1, -2, 0, 0, 0, 1, -1, 0, -1, 1, 0, -1, 0]
[0, -1, -1, 1, 0, -1, 1, -1, 0, 1, 1, -1, 1, -1, 1, 1, -1, 0, 1, 1, -1, 1, 0, -1, 2, 2, -1, -1, -1, 0, 1, 0, -1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 2

In [5]:
# checks c1 in ACCU1
# c1 = Encode(Compress(u)), u =InvNTT(A * y) + e1
# simulation time: 116685ps when ACCU1 contains C1 (and part of u)
from ipynb.fs.defs.matrix_hash import samplentt_suite
from ipynb.fs.defs.cbd2 import cbd2_hash
from ipynb.fs.defs.ntt_invntt import invntt, barrett_reduce
from ipynb.fs.defs.checking import check_dual
from ipynb.fs.defs.encode import compress_encode_10
from ipynb.fs.defs.test import convert_mem
from ipynb.fs.defs.compress import compress
from ipynb.fs.defs.polyvec import polyvec_basemul_acc_mont, poly_add, poly_reduce
from ipynb.fs.defs.pke_keygen import read_from_file

# m_matrix ="0708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f20212223242526" 
# m_hash = "02030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f2021"

m_matrix = string_list_conect(read_from_file("../sim_input/p_seed.hex", 0))
m_hash = string_list_conect(read_from_file("../sim_input/r_randomness.hex", 0))

AA = [[]]*3*3
Ay = [[]]*3
Intt_Ay = [[]]*3
accu1 = [[]]*3
e1 = [[]]*3
c1 = [[]]*3
mem_accu1 = [[]]*2*3

# matrix sample ntt
for i in range(3):
    for j in range(3):
        AA[i*3+j] = samplentt_suite(m_matrix,j,i)
        # print(i,j,AA[i][j])

# A * y
for i in range(3):
    Ay[i] = polyvec_basemul_acc_mont(AA[i*3+0], AA[i*3+1], AA[i*3+2], m_hash,0,1,2)
    print("A*y:",i, Ay[i])
    Intt_Ay[i] = invntt(Ay[i])
    # print(i, Ay[i][::-1])
    e1[i] = cbd2_hash(m_hash, i+4 )
    print("e1:",i,e1[i])
    print("InvNTT(A*y):",i,Intt_Ay[i])
    print()

for i in range(3):
    accu1[i] = (poly_add(e1[i], Intt_Ay[i]))
    accu1[i] = poly_reduce(accu1[i])

for i in range(3):
    c1[i] = compress_encode_10(accu1[i])

for i in range(3):
    mem_accu1[i*2 + 0] = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_a-k_"+str(i)+".txt")
    mem_accu1[i*2 + 1] = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_b-k_"+str(i)+".txt")
    # print(mem_accu1[i*2 + 0])
    # print(mem_accu1[i*2 + 1])

compressed_mem = [[]]*2*3
for i in range(3):
    for j in range(2):
        compressed_mem[i*2+j] = to_hex(mem_accu1[i*2 + j][0:80])
        print("c1:",i,"even or odd:",j,compressed_mem[i*2+j])

# test data produced by C code, this is compressed(u[0])
test = [445, 366, 360, 977,
892, 884, 765, 538,
105, 110, 982, 671,
835, 22, 505, 618,
107, 793, 657, 462,
30, 784, 188, 371,
231, 515, 504, 345,
542, 356, 635, 555,
54, 290, 100, 604,
233, 145, 612, 222,
15, 654, 501, 411,
459, 949, 420, 267,
400, 1008, 583, 385,
143, 175, 850, 370,
143, 506, 233, 260,
434, 683, 795, 689,
337, 530, 914, 113,
711, 747, 403, 554,
364, 142, 176, 731,
643, 832, 159, 592,
560, 217, 490, 57,
276, 2, 174, 769,
779, 542, 933, 369,
70, 746, 333, 541,
96, 665, 683, 861,
846, 41, 96, 54,
609, 30, 515, 906,
314, 939, 989, 436,
305, 14, 16, 491,
778, 933, 479, 822,
136, 703, 700, 918,
420, 864, 941, 500,
425, 35, 699, 305,
858, 410, 691, 1,
3, 329, 718, 31,
483, 126, 299, 268,
184, 876, 374, 599,
605, 865, 67, 862,
646, 445, 975, 265,
197, 32, 301, 544,
881, 416, 950, 0,
297, 186, 995, 879,
712, 997, 427, 261,
495, 116, 230, 415,
622, 81, 205, 849,
736, 1010, 435, 391,
621, 236, 62, 464,
578, 657, 970, 165,
494, 888, 628, 467,
883, 769, 883, 351,
107, 881, 419, 501,
843, 926, 439, 487,
755, 222, 110, 131,
258, 656, 160, 428,
619, 438, 87, 864,
807, 984, 635, 117,
832, 765, 820, 440,
212, 502, 1004, 663,
164, 94, 295, 149,
521, 99, 97, 948,
573, 743, 351, 544,
298, 772, 557, 400,
203, 777, 730, 654,
212, 205, 512, 513]

# checking it against my own result
# for i in range(len(accu1[0])):
#     t = compress(accu1[0][i],10,32)
#     if(test[i]!=t):
#         print("[+] proc")

for i in range(3):
    check_dual(c1[i],compressed_mem[i*2],compressed_mem[i*2+1])

print("[+] done")

A*y: 0 [36, 562, 540, 1490, -1157, -61, 1053, -987, -797, 463, -414, -1453, -4, 762, -327, 715, 1133, 981, 1150, -657, -563, -769, 1004, -1282, -1196, 607, -1603, -1094, -1028, 344, -724, 1405, -502, -10, -75, 1546, -847, 1653, -1651, 1179, 1494, 1274, 1332, 353, -1215, 987, 717, -199, 505, -1525, -497, 1363, -239, -71, -616, 231, -1279, -338, -23, -1570, -1248, 1413, 1513, -83, 1499, -1236, -51, 898, 1451, 509, 667, -277, -1239, -1645, 1660, 717, -294, -624, -1656, -817, -947, 993, -772, 1278, -824, -443, 1232, -502, -733, 1263, -1590, -564, 1017, 675, -1526, -964, 193, -72, -829, -1, 1320, 1490, 1487, 390, -130, 1448, 1422, 1429, 1242, 899, 10, 1495, 1598, 1583, -879, -1385, -1254, 1383, -765, -914, -774, 832, 303, -714, 321, 1282, -173, -932, -1469, 1243, -1456, 150, 703, 241, 1062, 254, -1169, 1215, 93, -27, -1029, 1289, 1448, 344, -424, -1165, -1191, -9, 1074, -1005, -1445, 490, -1267, 1387, -558, -861, -269, 18, -1659, 614, -312, -93, -1280, -1632, -1189, -1273, 496, -131, 1179, 

In [6]:
# checks c2 in ACCU2
# v = DECOMPRESS1(msg) + CBD(m,3) + InvNTT(t*y)
# t (vector) = encryption key t
# y (vector) = NTT(CBD(m,0)), NTT(CBD(m,1)), NTT(CBD(m,2))
# simulation time: 62465ps when ACCU2 contains c2 (and part of v)

from ipynb.fs.defs.decompress import decompress1_8_list
from ipynb.fs.defs.cbd2 import cbd2_hash
from ipynb.fs.defs.polyvec import polyvec_basemul_acc_mont, poly_add
from ipynb.fs.defs.ntt_invntt import invntt
from ipynb.fs.defs.test import convert_mem
from ipynb.fs.defs.encode import compress_encode_4
from ipynb.fs.defs.pke_keygen import read_from_file, decode12

# msg = [
#  5,  6,  7,  8,  9, 10, 11, 12,
# 13, 14, 15, 16, 17, 18, 19, 20,
# 21, 22, 23, 24, 25, 26, 27, 28,
# 29, 30, 31, 32, 33, 34, 35, 36 ]
# m_hash = "02030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f2021"
# ekt = [[]]*3
# for i in range(3):
#     temp = []
#     for j in range(256):
#         temp.append(i*256 + j)
#     ekt[i] = temp

msg = read_from_file("../sim_input/message.hex", 1)
print(msg)
m_hash = string_list_conect(read_from_file("../sim_input/r_randomness.hex", 0))
ek_PKE = decode12(read_from_file("../sim_input/encryption_key.hex", 1))
print(ek_PKE)
ekt = [[]]*3
for i in range(3):
    ekt[i] = ek_PKE[256*i:256*i+256]

# t*y
ty = polyvec_basemul_acc_mont(ekt[0],ekt[1],ekt[2],m_hash,0,1,2)
Intt_ty = invntt(ty)
e2 = cbd2_hash(m_hash,3)
mu = decompress1_8_list(msg)

v = poly_add(mu,e2)
v = poly_add(v,Intt_ty)

mem_accu2_a = convert_mem("../sim_output/tb_kyber_pke_enc-accu2-ram_a.txt")
mem_accu2_b = convert_mem("../sim_output/tb_kyber_pke_enc-accu2-ram_b.txt")

compressed_accu2_mem = [[]]*2
compressed_accu2_mem[0] = to_hex(mem_accu2_a[0:32])
compressed_accu2_mem[1] = to_hex(mem_accu2_b[0:32])

c2 = compress_encode_4(v)
print("c2:",c2)
check_dual(c2,compressed_accu2_mem[0],compressed_accu2_mem[1])

print("[+] done")


[50, 248, 180, 230, 143, 148, 244, 223, 85, 112, 196, 9, 213, 74, 14, 97, 124, 137, 68, 137, 141, 30, 244, 143, 137, 232, 10, 85, 119, 141, 62, 87]
[952, 255, 3281, 1478, 3270, 1541, 359, 2007, 2286, 740, 2517, 1613, 2268, 191, 279, 2397, 396, 448, 1506, 3125, 1306, 848, 1493, 926, 2126, 2995, 602, 3083, 2442, 1327, 608, 868, 357, 360, 2123, 2067, 981, 3149, 252, 348, 1309, 470, 1424, 892, 3056, 2494, 375, 1329, 1031, 41, 2922, 1344, 2294, 3036, 2656, 2245, 841, 1740, 761, 2280, 2878, 1748, 603, 2331, 2436, 2491, 2499, 1179, 1989, 1730, 2275, 1496, 2316, 625, 994, 972, 3226, 158, 2708, 601, 861, 221, 370, 2554, 1066, 1766, 2504, 499, 512, 136, 374, 628, 655, 1335, 2215, 3178, 2902, 2523, 1737, 938, 783, 2529, 3187, 2879, 2039, 1269, 153, 2605, 1786, 2433, 2019, 1716, 174, 1158, 673, 1695, 1157, 3078, 323, 1564, 3285, 2283, 2087, 1147, 1056, 2852, 798, 2543, 1991, 3200, 2663, 814, 2991, 956, 178, 1478, 1234, 1643, 549, 1679, 734, 3122, 3271, 2144, 686, 2539, 118, 511, 3091, 2944, 114, 1

In [9]:
from ipynb.fs.defs.test import convert_mem
from ipynb.fs.defs.polyvec import check

c_mem = convert_mem("../sim_output/tb_kyber_pke_enc-c_mem.txt")
c_mem = to_hex(c_mem)
print("len of c", len(c_mem))
print("cipher text", c_mem)

print("[+] c1")
for i in range(3):
    check(c1[i],c_mem[160*i + 0: 160*i + 160])

print("[+] c2")
check(c2, c_mem[480:544])

len of c 544
cipher text ['0xfac0', '0xb1fc', '0x90a5', '0xd739', '0x8b95', '0x1689', '0x46e9', '0x28e9', '0x6acd', '0x1905', '0x2d9f', '0xb915', '0x5f06', '0x50d4', '0x2a0b', '0x5dad', '0x64c4', '0x796a', '0xdb3d', '0xa58e', '0x2691', '0xe785', '0xa19d', '0x4a24', '0x6625', '0xa528', '0x963', '0x6a9a', '0xe866', '0xd998', '0xee05', '0x12b7', '0xe619', '0x9d4e', '0x31b8', '0x9ac9', '0x630', '0x747', '0xb70', '0xfbde', '0xd0da', '0x9d49', '0x97dc', '0x5120', '0x140f', '0xceca', '0xf6cb', '0x5c20', '0x2ccf', '0x7cfa', '0x2fb2', '0x8bb2', '0xcda4', '0xbf4e', '0xe59a', '0x5919', '0xabc9', '0x199a', '0xef04', '0x32e6', '0x5264', '0xb435', '0x7fc4', '0x1058', '0x7cf4', '0x9f96', '0x86c5', '0x6731', '0x173a', '0x6ce9', '0x70a3', '0xb076', '0xea', '0xa933', '0x550f', '0x3812', '0xd55a', '0xe855', '0xf401', '0x93f0', '0xd9ef', '0xa58e', '0x714b', '0x9abd', '0x3e71', '0xf990', '0x216d', '0x994a', '0xfad4', '0xaeed', '0xe27d', '0xabf8', '0xfa85', '0x6301', '0x3e5f', '0xb0d8', '0xe28d', '0xcc27', 